In [1]:
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
import numpy as np

import trimesh
import pyrender
import pdb
import re
import math

from skimage.transform import rescale, resize
from scipy.ndimage import median_filter

def surface_normal(points, sH, sW):
    # These lookups denote y,x offsets from the anchor point for 8 surrounding
    # directions from the anchor A depicted below.
    #  -----------
    # | 7 | 6 | 5 |
    #  -----------
    # | 0 | A | 4 |
    #  -----------
    # | 1 | 2 | 3 |
    #  -----------
    d = 2
#     lookups = {0:(-d,0),1:(-d,d),2:(0,d),3:(d,d),4:(d,0),5:(d,-d),6:(0,-d),7:(-d,-d)}

    lookups = {0:(0,-d),1:(d,-d),2:(d,0),3:(d,d),4:(0,d),5:(-d,d),6:(-d,0),7:(-d,-d)}

    surface_normals = np.zeros((sH,sW,3))
    for i in range(sH):
        for j in range(sW):
            min_diff = None
            point1 = points[i,j,:3]
             # We choose the normal calculated from the two points that are
             # closest to the anchor points.  This helps to prevent using large
             # depth disparities at surface borders in the normal calculation.
            for k in range(8):
                try:
                    point2 = points[i+lookups[k][0],j+lookups[k][1],:3]
                    point3 = points[i+lookups[(k+2)%8][0],j+lookups[(k+2)%8][1],:3]
                    diff = np.linalg.norm(point2 - point1) + np.linalg.norm(point3 - point1)
                    if min_diff is None or diff < min_diff:
                        normal = normalize(np.cross(point2-point1,point3-point1))
                        min_diff = diff
                except IndexError:
                    continue
            surface_normals[i,j,:3] = normal
    return surface_normals

def normalize(v):
    return v/np.linalg.norm(v)

def depth_to_surface_normal_opencv_projection(depth, intrinsics, extrinsics, cls, seq, center, scale=0.25):
    depth_map = depth.copy()
    H, W = depth.shape
    sH, sW = int(scale*H), int(scale*W)
    depth_map[depth < 0.0001] = 50.0

    # Each 'pixel' containing the 3D point in camera coords
    depth_in_world = depth2world(depth_map, intrinsics, extrinsics, cls, seq, center, True)[:,:3].reshape(H,W,3)
    surface_normals = surface_normal(depth_in_world[::int(1/scale),::int(1/scale),:], sH, sW)
    surface_normals = resize(surface_normals, (H, W), anti_aliasing=True)
    return surface_normals

def R_z(theta):
    theta = theta/180*np.pi
    return np.array([[np.cos(theta), -np.sin(theta), 0],
                    [np.sin(theta), np.cos(theta), 0],
                    [0, 0,1]])
def R_y(theta):
    theta = theta/180*np.pi
    return np.array([[np.cos(theta), 0, np.sin(theta)],
                    [0, 1, 0],
                    [-np.sin(theta),0,np.cos(theta)]])
def R_x(theta):
    theta = theta/180*np.pi
    return np.array([[1, 0, 0],
                    [0, np.cos(theta), -np.sin(theta)],
                    [0,np.sin(theta), np.cos(theta)]])

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def quat2rot(quat):
    mtx = np.zeros((4,4))
    mtx[3,3] = 1
    r = R.from_quat(quat[3:])
    rot = r.as_matrix()
    mtx[:3,:3] = rot
    mtx[:3,3] =  quat[:3]
    return mtx

def ext_test(cam_ext_file, time):
    # ['/map', 'CreditCardWith4Markers']
    # ['/map', 'AsusXtionCameraFrame']
    # ['/map', 'TransparentBoxWith4Markers']

    # ['/AsusXtionCameraFrame', '/camera_link']
    # ['/camera_link', '/camera_rgb_frame']
    # ['/camera_link', '/camera_depth_frame']
    # ['/camera_depth_frame', '/camera_depth_optical_frame']

    quat = np.zeros(7)

    Asus2cam_quat_list = list()
    cam2depth_quat_list = list()
    cam2rgb_quat_list = list()
    depth2optical_quat_list = []
    map2Asus_quat_list = []

    Asus2cam_time_list = []
    cam2depth_time_list = []
    cam2rgb_time_list = []
    depth2optical_time_list = []
    map2Asus_time_list = []

    cams = ['field.transforms0.header.frame_id', 'field.transforms0.child_frame_id']
    
    pose = ['field.transforms0.transform.translation.x','field.transforms0.transform.translation.y',
            'field.transforms0.transform.translation.z','field.transforms0.transform.rotation.x',
            'field.transforms0.transform.rotation.y','field.transforms0.transform.rotation.z',
            'field.transforms0.transform.rotation.w']
    
    with open(cam_ext_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            for i, p in enumerate(pose):
                quat[i] = float(row[p])

            if row[cams[0]]=='/AsusXtionCameraFrame' and row[cams[1]]=='/camera_link':
                Asus2cam_quat_list.append(np.array(list(quat)))
                Asus2cam_time_list.append(float(row['%time']))
            elif row[cams[0]]=='/camera_link' and row[cams[1]]=='/camera_depth_frame':
                cam2depth_quat_list.append(np.array(list(quat)))
                cam2depth_time_list.append(float(row['%time']))
            elif row[cams[0]]=='/camera_link' and row[cams[1]]=='/camera_rgb_frame':
                cam2rgb_quat_list.append(np.array(list(quat)))
                cam2rgb_time_list.append(float(row['%time']))
            elif row[cams[0]]=='/camera_depth_frame' and row[cams[1]]=='/camera_depth_optical_frame':
                depth2optical_quat_list.append(np.array(list(quat)))
                depth2optical_time_list.append(float(row['%time']))
            elif row[cams[0]]=='/map' and row[cams[1]]=='AsusXtionCameraFrame':
                map2Asus_quat_list.append(np.array(list(quat)))
                map2Asus_time_list.append(float(row['%time']))
            else:
                continue
            

    idx = find_nearest(Asus2cam_time_list, time)
    Asus2cam = quat2rot(Asus2cam_quat_list[idx])
    
    idx = find_nearest(cam2depth_time_list, time)
    cam2depth = quat2rot(cam2depth_quat_list[idx])
    
    idx = find_nearest(cam2rgb_time_list, time)
    cam2rgb = quat2rot(cam2rgb_quat_list[idx])
    
    idx = find_nearest(depth2optical_time_list, time)
    depth2optical = quat2rot(depth2optical_quat_list[idx])
    
    idx = find_nearest(map2Asus_time_list, time)
    map2Asus = quat2rot(map2Asus_quat_list[idx])
    
    map2optical = map2Asus@Asus2cam@cam2depth@depth2optical

    return np.linalg.inv(map2optical)

        
def pose_test(pose_file, time):
    marker0_list = np.zeros(7)
    marker1_list = np.zeros(7)
    marker2_list = np.zeros(7)

    marker0 = np.zeros(7)
    marker1 = np.zeros(7)
    marker2 = np.zeros(7)
    
    time_list = []

    pose = ['pose.position.x', 'pose.position.y', 'pose.position.z',
                     'pose.orientation.x', 'pose.orientation.y', 'pose.orientation.z', 'pose.orientation.w']

    with open(pose_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for j, row in enumerate(reader):
            if j<635:
                for i, p in enumerate(pose):

                    marker0[i] = float(row['field.markers0.' + p])
                    marker1[i] = float(row['field.markers1.' + p])
                    marker2[i] = float(row['field.markers2.' + p])
                marker0_list = np.vstack((marker0_list, marker0))
                marker1_list = np.vstack((marker1_list, marker1))
                marker2_list = np.vstack((marker2_list, marker2))
                time_list.append(float(row['%time']))
        marker0_list = marker0_list[1:,:]
        marker1_list = marker1_list[1:,:]
        marker2_list = marker2_list[1:,:]

    idx = find_nearest(time_list, time)
    map2obj = quat2rot(marker2_list[idx])
    
    return map2obj


def int_test(cam_int_file):
    
    K = np.zeros(9)
    K_items = ['field.K0', 'field.K1', 'field.K2', 'field.K3', 'field.K4', 'field.K5',
         'field.K6', 'field.K7', 'field.K8']
    
    with open(cam_int_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            for i, K_item in enumerate(K_items):
                K[i] = float(row[K_item])
            return K.reshape(3,3)

def js_test(js_int_file, time):

    time_list = []
    js_list = []
    
    with open(js_int_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            js = int(float(row['field.position0'])/np.pi*180)
            t = float(row['%time'])
            js_list.append(js)
            time_list.append(t)
        idx = find_nearest(time_list, time)
    return js_list[idx]
    
def depth2world(depth_map, intrinsic_param, extrinsic_param, cls, seq, center, return_full=False):

    # Get world coords
    H, W = depth_map.shape

    WS = np.repeat(np.linspace(1 / (2 * W), 1 - 1 / (2 * W), W).reshape([1, -1]), H, axis=0)
    HS = np.repeat(np.linspace(1 / (2 * H), 1 - 1 / (2 * H), H).reshape([-1, 1]), W, axis=1)

    pixel_coords = np.stack([WS*W, HS*H, np.ones(depth_map.shape)], 2)
    pixel_coords = pixel_coords.reshape(-1, 3).T
    depth_map = depth_map.reshape(-1,1).T
    
    cam_coords = np.linalg.inv(intrinsic_param)@(pixel_coords)
    cam_coords *= depth_map
    
    cam_coords = np.vstack([cam_coords, np.ones((1,cam_coords.shape[1]))])
    world_coords = np.linalg.inv(extrinsic_param)@cam_coords
    
    world_coords = world_coords.T

    if return_full==False:
        mask = np.repeat(depth_map.copy(), 4, axis=0).T
        world_coords = world_coords[mask>0].reshape(-1,4)
        world_coords = alignment(cls, seq, center, world_coords)
    else:
        world_coords = alignment(cls, seq, center, world_coords)

    return world_coords


# specify alignment
def alignment(cls, seq, center, world_coords):
    align = np.eye(4)

    # scale to center
#     if center is None:
#         c = np.mean(world_coords, axis = 0)
#     else:
#         c = center[seq]
    if seq not in center.keys():
        c = np.mean(world_coords, axis = 0)
        center[seq] = c
    else:
        c = center[seq]
    print(np.mean(world_coords, axis = 0))
    print("c: ", c)
    world_coords = world_coords - c
#     scale = 1/np.max(np.abs(world_coords))
    
    if cls == 'laptop':
        world_coords *= 2.5
        world_coords = world_coords.T
        x = -5
        y = 310
        z = 120
        align[:3,:3] = R_z(z)@R_y(y)@R_x(x)
        world_coords = align@world_coords
        world_coords = world_coords.T
        world_coords[:,0] += 0.1
        world_coords[:,1] += 0.0
        world_coords[:,2] -= 0.25

    return world_coords


# Generate real dataset

In [3]:
import glob
import os
import re


def generate_SDF(depth, mask, intrinsic_param, extrinsic_param, pose, cls, seq, center, eta=0.025):

#     depth, mask, intrinsic_param, extrinsic_param, pose = get_default_scene(filename, cls)
    H, W = depth.shape
    depth[depth>1] = 0 
    depth = median_filter(depth, size=15)

    world_coords = depth2world(depth, intrinsic_param, extrinsic_param, cls, seq, center)

    surface_normals = depth_to_surface_normal_opencv_projection(depth, intrinsic_param, extrinsic_param, cls, seq, center)

    WS = np.repeat(np.linspace(1 / (2 * W), 1 - 1 / (2 * W), W).reshape([1, -1]), H, axis=0) * W
    HS = np.repeat(np.linspace(1 / (2 * H), 1 - 1 / (2 * H), H).reshape([-1, 1]), W, axis=1) * H
    s_X = WS[depth > 0]
    s_Y = HS[depth > 0]

    pos_pts_world = world_coords[:,:3].copy()
    neg_pts_world = world_coords[:,:3].copy()
    # purturb with surface normal
    for idx, (xx,yy) in enumerate(zip(s_X,s_Y)):
        pos_pts_world[idx] += eta * np.array(surface_normals[int(yy)][int(xx)])
        neg_pts_world[idx] -= eta * np.array(surface_normals[int(yy)][int(xx)])

    eta_vec = np.ones((pos_pts_world.shape[0], 1)) * eta
    part = np.zeros((pos_pts_world.shape[0], 1))
    pos = np.hstack([pos_pts_world, eta_vec, part])
    neg = np.hstack([neg_pts_world, -eta_vec, part])
    
    return pos, neg, world_coords[:,:3]


def generate_testset(save_dir, cls='oven'):
    
    file_dir_ = '/media/jm/000B48300008D6EB/dataset/real_laptop/'
    seq_dir = ['laptop01_o', 'laptop02_o', 'laptop03_o', 'laptop04_o', 'laptop05_o', 
               'laptop06_o', 'laptop07_o', 'laptop08_o', 'laptop09_o', 'laptop10_o', 
               'laptop11_o', 'laptop12_o', 'laptop13_o', 'laptop14_o',
               'laptop16_o', 'laptop17_o', 'laptop18_o', 'laptop19_o', 'laptop20_o'] # seq 15 diff annotation
    
    center = {}

    test_files = {
        'laptop01_o': ['000114', '000112', '000110', '000108', '000106', '000103', '000100', '000097', '000094', '000091'],
        'laptop02_o': ['000091', '000094', '000097', '000100', '000103', '000106', '000109', '000115', '000120', '000125'],
        'laptop03_o': ['000091', '000088', '000085', '000082', '000079', '000076', '000073', '000070', '000067', '000064'],
        'laptop04_o': ['000137', '000136', '000134', '000131', '000128', '000125', '000122', '000119', '000116', '000114'],        
        'laptop05_o': ['000101', '000104', '000107', '000110', '000113', '000116', '000118', '000120', '000125', '000130'],
        'laptop06_o': ['000091', '000094', '000097', '000100', '000103', '000106', '000109', '000112', '000115', '000118'],
        'laptop07_o': ['000076', '000073', '000070', '000067', '000064', '000061', '000058', '000055', '000052', '000049'],
        'laptop08_o': ['000094', '000097', '000100', '000103', '000106', '000109', '000112', '000115', '000118', '000120'],
        'laptop09_o': ['000108', '000106', '000104', '000102', '000100', '000098', '000095', '000092', '000089', '000085'],
        'laptop10_o': ['000109', '000111', '000113', '000115', '000117', '000119', '000121', '000123', '000125', '000127'],
        'laptop11_o': ['000216', '000214', '000212', '000211', '000210', '000209', '000206', '000203', '000200', '000197'],
        'laptop12_o': ['000025', '000028', '000031', '000034', '000037', '000040', '000043', '000046', '000050', '000055'],
        'laptop13_o': ['000055', '000061', '000064', '000067', '000070', '000073', '000076', '000079', '000082', '000085'],
        'laptop14_o': ['000121', '000120', '000119', '000118', '000115', '000112', '000109', '000106', '000103', '000100'],
        'laptop16_o': ['000094', '000097', '000100', '000104', '000106', '000108', '000110', '000114', '000118', '000122'],
        'laptop17_o': ['000142', '000139', '000134', '000132', '000130', '000127', '000124', '000121', '000118', '000115'],
        'laptop18_o': ['000130', '000133', '000136', '000139', '000142', '000145', '000148', '000151', '000154', '000157'],
        'laptop19_o': ['000077', '000080', '000083', '000086', '000089', '000092', '000095', '000098', '000101', '000103'],
        'laptop20_o': ['000090', '000093', '000096', '000100', '000103', '000106', '000109', '000112', '000115', '000118'],
    }

    dic = {'real':{'shape2motion':[]}}
    for seq in seq_dir:
        file_dir = file_dir_ + seq
        pose_file = file_dir + '/rb_poses_array.csv' # 795
        # cam_int_file = file_dir + '/camera_rgb_camera_info.csv' # 236
        cam_int_file = file_dir + '/camera_depth_registered_camera_info.csv' # 236
        cam_ext_file = file_dir + '/tf.csv' # 2730
        js_file = file_dir + '/laptop_joint_states.csv' # 2730
        
        for file in test_files[seq]:

            depth_file = glob.glob(os.path.join(file_dir, 'camera_depth_registered/', file + '*'))[0]
            mask_file = glob.glob(os.path.join(file_dir, 'camera_rgb/', file + '*' + '_mask.png'))[0]
            time_idx = float(re.split('/', depth_file)[-1][7:21])*1e9
            
            print(depth_file)

            mask = plt.imread(mask_file)

            real_depth = np.loadtxt(depth_file)
            real_depth = np.nan_to_num(real_depth)
            real_depth[mask==0] = 0

            # get camera params
            optical_T_map = ext_test(cam_ext_file, time_idx)
            map_T_obj = pose_test(pose_file, time_idx)

            extrinsic_param = optical_T_map@map_T_obj

            intrinsic_param = int_test(cam_int_file)
            
            js = js_test(js_file, time_idx)
            print('{:04d}'.format(js))

            pos, neg, gt = generate_SDF(real_depth, mask, intrinsic_param, extrinsic_param, map_T_obj, cls, seq, center)
            if cls=='laptop':
                np.savez(os.path.join(save_dir, 'sdf', '00'+seq[:-1][6:9]+file+'_{:04d}'.format(js)), pos=pos, neg=neg)
                np.savez(os.path.join(save_dir, 'gt', '00'+seq[:-1][6:9]+file+'_{:04d}'.format(js)), pos=gt, neg=gt)
            elif cls=='oven':
                np.savez(os.path.join(save_dir, 'sdf', '00'+seq[:-1][9:13]+file+'_{:04d}'.format(js)), pos=pos, neg=neg)
                np.savez(os.path.join(save_dir, 'gt', '00'+seq[:-1][9:13]+file+'_{:04d}'.format(js)), pos=gt, neg=gt)

            dic['real']['shape2motion'].append('00'+seq[:-1][6:9]+file+'_{:04d}'.format(js)) # laptop
    import json
    with open("rbo_laptop/sm_rbo_laptop_6_angle_test.json", "w") as f:
        json.dump(dic, f)

save_dir = 'rbo_laptop'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(save_dir+'/sdf'):
    os.makedirs(save_dir+'/sdf')
if not os.path.exists(save_dir+'gt'):
    os.makedirs(save_dir+'/gt')

generate_testset(save_dir)

/media/jm/000B48300008D6EB/dataset/real_laptop/laptop01_o/camera_depth_registered/000114-1496323073.207.txt
-059
[ 0.0836119  -0.09474277 -0.07062009  1.        ]
c:  [ 0.0836119  -0.09474277 -0.07062009  1.        ]
[ -6.26376305  36.09535015 -28.5201722    1.        ]
c:  [ 0.0836119  -0.09474277 -0.07062009  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop01_o/camera_depth_registered/000112-1496323073.141.txt
-055
[ 0.09971233 -0.08374545 -0.06396342  1.        ]
c:  [ 0.0836119  -0.09474277 -0.07062009  1.        ]
[ -5.98692214  35.41724951 -27.92208186   1.        ]
c:  [ 0.0836119  -0.09474277 -0.07062009  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop01_o/camera_depth_registered/000110-1496323073.074.txt
-051
[ 0.10328805 -0.07921217 -0.06417633  1.        ]
c:  [ 0.0836119  -0.09474277 -0.07062009  1.        ]
[ -5.93522236  35.10129784 -27.6330089    1.        ]
c:  [ 0.0836119  -0.09474277 -0.07062009  1.        ]
/media/jm/000B48300008D6E

-022
[ 0.17443283 -0.03229811 -0.09784258  1.        ]
c:  [ 0.15783926 -0.05461725 -0.09566859  1.        ]
[-19.19654535  39.5410763   -9.05516418   1.        ]
c:  [ 0.15783926 -0.05461725 -0.09566859  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop03_o/camera_depth_registered/000073-1496323090.092.txt
-022
[ 0.18087251 -0.03463018 -0.10209801  1.        ]
c:  [ 0.15783926 -0.05461725 -0.09566859  1.        ]
[-19.90136291  39.67609819  -8.26419663   1.        ]
c:  [ 0.15783926 -0.05461725 -0.09566859  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop03_o/camera_depth_registered/000070-1496323089.992.txt
-022
[ 0.18675768 -0.03526498 -0.1034672   1.        ]
c:  [ 0.15783926 -0.05461725 -0.09566859  1.        ]
[-20.06700856  39.83209537  -8.54996631   1.        ]
c:  [ 0.15783926 -0.05461725 -0.09566859  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop03_o/camera_depth_registered/000067-1496323089.892.txt
-022
[ 0.1923343  -0.0320

/media/jm/000B48300008D6EB/dataset/real_laptop/laptop06_o/camera_depth_registered/000094-1496323170.910.txt
-054
[ 0.13634105 -0.16196059 -0.15957196  1.        ]
c:  [ 0.14894372 -0.15948111 -0.15682276  1.        ]
[-31.41429931  33.97478968   7.63896784   1.        ]
c:  [ 0.14894372 -0.15948111 -0.15682276  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop06_o/camera_depth_registered/000097-1496323171.010.txt
-044
[ 0.14291484 -0.14307692 -0.13680716  1.        ]
c:  [ 0.14894372 -0.15948111 -0.15682276  1.        ]
[-31.35207106  33.66905997   6.97944633   1.        ]
c:  [ 0.14894372 -0.15948111 -0.15682276  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop06_o/camera_depth_registered/000100-1496323171.110.txt
-035
[ 0.12579156 -0.14359125 -0.15339568  1.        ]
c:  [ 0.14894372 -0.15948111 -0.15682276  1.        ]
[-31.26880517  33.80086836   7.48456351   1.        ]
c:  [ 0.14894372 -0.15948111 -0.15682276  1.        ]
/media/jm/000B48300008D6E

-030
[ 0.21477523 -0.01853794 -0.07106312  1.        ]
c:  [ 0.12739769 -0.11889562 -0.1129773   1.        ]
[-21.45506063  40.97626017  -7.34013922   1.        ]
c:  [ 0.12739769 -0.11889562 -0.1129773   1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop08_o/camera_depth_registered/000115-1496323240.617.txt
-031
[ 0.20742759 -0.02377951 -0.07943526  1.        ]
c:  [ 0.12739769 -0.11889562 -0.1129773   1.        ]
[-21.45409943  40.91949011  -7.02315376   1.        ]
c:  [ 0.12739769 -0.11889562 -0.1129773   1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop08_o/camera_depth_registered/000118-1496323240.717.txt
-030
[ 0.20993238 -0.01819284 -0.07574471  1.        ]
c:  [ 0.12739769 -0.11889562 -0.1129773   1.        ]
[-21.26641865  40.88799373  -7.40937018   1.        ]
c:  [ 0.12739769 -0.11889562 -0.1129773   1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop08_o/camera_depth_registered/000120-1496323240.783.txt
-031
[ 0.21760643 -0.0223

/media/jm/000B48300008D6EB/dataset/real_laptop/laptop11_o/camera_depth_registered/000212-1496323305.651.txt
-041
[ 0.0604399  -0.10754106 -0.05904698  1.        ]
c:  [ 0.05489662 -0.11349853 -0.06777551  1.        ]
[ -6.09557613  29.77528386 -32.3285644    1.        ]
c:  [ 0.05489662 -0.11349853 -0.06777551  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop11_o/camera_depth_registered/000211-1496323305.618.txt
-039
[ 0.05829832 -0.10802879 -0.06336672  1.        ]
c:  [ 0.05489662 -0.11349853 -0.06777551  1.        ]
[ -6.03163331  29.75395597 -32.45970619   1.        ]
c:  [ 0.05489662 -0.11349853 -0.06777551  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop11_o/camera_depth_registered/000210-1496323305.585.txt
-037
[ 0.05509862 -0.10954014 -0.07255179  1.        ]
c:  [ 0.05489662 -0.11349853 -0.06777551  1.        ]
[ -6.20088939  29.84589049 -32.5615113    1.        ]
c:  [ 0.05489662 -0.11349853 -0.06777551  1.        ]
/media/jm/000B48300008D6E

-017
[ 0.14208897 -0.04115632 -0.08432181  1.        ]
c:  [ 0.10955632 -0.1012393  -0.07538883  1.        ]
[-11.9179267   37.52589974 -16.47933137   1.        ]
c:  [ 0.10955632 -0.1012393  -0.07538883  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop13_o/camera_depth_registered/000082-1496323402.553.txt
-012
[ 0.14068289 -0.03628258 -0.08639166  1.        ]
c:  [ 0.10955632 -0.1012393  -0.07538883  1.        ]
[-11.96227505  37.55258508 -16.47122736   1.        ]
c:  [ 0.10955632 -0.1012393  -0.07538883  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop13_o/camera_depth_registered/000085-1496323402.653.txt
-010
[ 0.13540633 -0.03731056 -0.08637391  1.        ]
c:  [ 0.10955632 -0.1012393  -0.07538883  1.        ]
[-11.95937378  37.60090064 -16.57779989   1.        ]
c:  [ 0.10955632 -0.1012393  -0.07538883  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop14_o/camera_depth_registered/000121-1496323421.073.txt
-047
[ 0.09612521 -0.1053

/media/jm/000B48300008D6EB/dataset/real_laptop/laptop17_o/camera_depth_registered/000132-1496323493.349.txt
-041
[ 0.19608436 -0.00987877 -0.0970683   1.        ]
c:  [ 0.16620252 -0.0437282  -0.10774548  1.        ]
[-24.35698412 -35.85822506   7.76330112   1.        ]
c:  [ 0.16620252 -0.0437282  -0.10774548  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop17_o/camera_depth_registered/000130-1496323493.282.txt
-036
[ 2.03124302e-01 -8.07942131e-04 -1.00176086e-01  1.00000000e+00]
c:  [ 0.16620252 -0.0437282  -0.10774548  1.        ]
[-24.04380846 -35.73016543   8.01176154   1.        ]
c:  [ 0.16620252 -0.0437282  -0.10774548  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop17_o/camera_depth_registered/000127-1496323493.182.txt
-027
[ 0.21130056  0.01355414 -0.09916153  1.        ]
c:  [ 0.16620252 -0.0437282  -0.10774548  1.        ]
[-23.9672367  -35.85248218   7.93419564   1.        ]
c:  [ 0.16620252 -0.0437282  -0.10774548  1.        ]
/media/jm

-039
[ 0.20930592 -0.02038161 -0.10661985  1.        ]
c:  [ 0.17203122 -0.06796497 -0.10903016  1.        ]
[-33.49665097 -24.82179922  12.43030422   1.        ]
c:  [ 0.17203122 -0.06796497 -0.10903016  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop19_o/camera_depth_registered/000103-1496323529.820.txt
-036
[ 0.21230537 -0.01676668 -0.10875646  1.        ]
c:  [ 0.17203122 -0.06796497 -0.10903016  1.        ]
[-33.63697425 -24.84621432  12.50587669   1.        ]
c:  [ 0.17203122 -0.06796497 -0.10903016  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop20_o/camera_depth_registered/000090-1496323559.051.txt
-083
[ 0.14434692 -0.05543234 -0.11041255  1.        ]
c:  [ 0.14434692 -0.05543234 -0.11041255  1.        ]
[-24.00020086 -34.90814654  -7.35629039   1.        ]
c:  [ 0.14434692 -0.05543234 -0.11041255  1.        ]
/media/jm/000B48300008D6EB/dataset/real_laptop/laptop20_o/camera_depth_registered/000093-1496323559.151.txt
-078
[ 0.15271798 -0.0529